In [2]:
import cffi
import numba
import numba.core.typing.cffi_utils as cffi_support
import numpy as np

import ufl
from dolfinx.cpp.fem import Form_float64
from dolfinx.jit import ffcx_jit
from dolfinx import fem, mesh, io

from mpi4py import MPI
from petsc4py import PETSc

In [3]:
L = 1
W = 0.1
rho = 1
delta = W/L
gamma = 0.4*delta**2
beta = 1.25
mu = 1
lambda_ = beta
g = gamma

# Create mesh and define function space
domain = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, 0.0), (L, W)), n=(1, 1),
                            cell_type=mesh.CellType.triangle,)


In [4]:
Ue = ufl.VectorElement("Lagrange", domain.ufl_cell(), 1)
Se = ufl.TensorElement("DG", domain.ufl_cell(), 1, symmetry=True)

U = fem.FunctionSpace(domain, Ue)
S = fem.FunctionSpace(domain, Se)
DG0 = fem.FunctionSpace(domain, ('DG', 0))
CG1 = fem.FunctionSpace(domain, ('CG', 1))
CG2 = fem.FunctionSpace(domain, ('CG', 2))

In [5]:
# Get local dofmap sizes for later local tensor tabulations
Ssize = S.element.space_dimension
Usize = U.element.space_dimension

sigma, tau = ufl.TrialFunction(S), ufl.TestFunction(S)
u, v = ufl.TrialFunction(U), ufl.TestFunction(U)

mu = fem.Function(DG0)
lambda_ = fem.Function(DG0)

# mu.x.array[:] = np.full(len(mu.x.array), 1)
# lambda_.x.array[:] = np.full(len(lambda_.x.array), beta)

mu.vector.set(1)
lambda_.vector.set(beta)

# mu.x.scatter_forward()
# lambda_.x.scatter_forward()

In [6]:
dxm = ufl.Measure(
    "dx",
    domain=domain,
    metadata={"quadrature_degree": 2, "quadrature_scheme": "default"},
)

left_facets = mesh.locate_entities_boundary(domain, dim=1,
                                       marker=lambda x: np.isclose(x[0], 0.0))

dofs = fem.locate_dofs_topological(V=U, entity_dim=1, entities=left_facets)
bc = fem.dirichletbc(value=fem.Constant(domain, (PETSc.ScalarType(0), PETSc.ScalarType(0))), dofs=dofs, V=U)                            

In [7]:
def epsilon(u):
    return ufl.sym(ufl.grad(u))

def sigma(u):
    return lambda_*ufl.div(u)*ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

# Define variational problem
f = fem.Constant(domain, (PETSc.ScalarType(0), PETSc.ScalarType(-rho*g)))
b = ufl.inner(f, v)*ufl.dx

a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx

# JIT compile individual blocks tabulation kernels
nptype = "float64"
ffcxtype = "double"
ufcx_form, _, _ = ffcx_jit(domain.comm, a, form_compiler_params={"scalar_type": ffcxtype})
kernel = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")

ffi = cffi.FFI()
cffi_support.register_type(ffi.typeof("double"), numba.types.float64)
c_signature = numba.types.void(
    numba.types.CPointer(numba.typeof(PETSc.ScalarType())),
    numba.types.CPointer(numba.typeof(PETSc.ScalarType())),
    numba.types.CPointer(numba.typeof(PETSc.ScalarType())),
    numba.types.CPointer(numba.types.double),
    numba.types.CPointer(numba.types.int32),
    numba.types.CPointer(numba.types.uint8))

In [10]:
@numba.cfunc(c_signature, nopython=True)
def tabulate_tensor_A(A_, w_, c_, coords_, entity_local_index, permutation=ffi.NULL):
    # Prepare target condensed local elem tensor
    # A = numba.carray(A_, (Usize, Usize), dtype=PETSc.ScalarType)
    # Tabulate all sub blocks locally
    # A00 = np.zeros((Usize, Usize), dtype=PETSc.ScalarType)
    
    w_[0] = beta #lambda
    w_[1] = 1 #mu
    c_[0] #fem.Constant f == [0, -rho*g]
    kernel(A_, w_, c_, coords_, entity_local_index, permutation)

    # A[:, :] = A00


In [11]:
integrals = {fem.IntegralType.cell: ([(-1, tabulate_tensor_A.address)], None)}
a_cond = Form_float64([U._cpp_object, U._cpp_object], integrals, [lambda_._cpp_object, mu._cpp_object], [f._cpp_object], False, None)

A_cond = fem.petsc.assemble_matrix(a_cond, bcs=[bc])
A_cond.assemble()
print(A_cond[:,:])

b_assembled = fem.petsc.assemble_vector(fem.form(b))
fem.petsc.apply_lifting(b_assembled, [a_cond], bcs=[[bc]])
b_assembled.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
fem.petsc.set_bc(b_assembled, [bc])

uc = fem.Function(U)
solver = PETSc.KSP().create(A_cond.getComm())
solver.setOperators(A_cond)

# It gives a different result, if we remove next two lines
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

solver.solve(b_assembled, uc.vector)
uc.x.scatter_forward()

[[  1.       0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       1.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       5.1625  -1.125   -5.       0.625    0.       0.    ]
 [  0.       0.      -1.125   16.3      0.5    -16.25     0.       0.    ]
 [  0.       0.      -5.       0.5      5.1625   0.       0.       0.    ]
 [  0.       0.       0.625  -16.25     0.      16.3      0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       1.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.       1.    ]]


In [12]:
problem = fem.petsc.LinearProblem(a, b, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [13]:
uh2 = fem.Function(U)

A = fem.petsc.assemble_matrix(fem.form(a), bcs=[bc])
A.assemble()

B = fem.petsc.assemble_vector(fem.form(b))

# A = fem.petsc.create_matrix(fem.form(a00))
# B = fem.petsc.create_vector(fem.form(b))

# A.zeroEntries()
# fem.petsc.assemble_matrix(A, fem.form(a00), bcs=[bc])
# A.assemble()

# with B.localForm() as B_local:
#     B_local.set(0.0)
# fem.petsc.assemble_vector(B, fem.form(b))

fem.apply_lifting(B, [fem.form(a)], bcs=[[bc]])
B.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
fem.set_bc(B, [bc])

solver = PETSc.KSP().create(A.getComm())
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)
solver.solve(B, uh2.vector)

uh2.x.scatter_forward()

In [15]:
print(A[:,:] - A_cond[:,:])

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [16]:
import ctypes
import ctypes.util
import petsc4py.lib
from petsc4py import get_config as PETSc_get_config

In [17]:
# Get details of PETSc install
petsc_dir = PETSc_get_config()['PETSC_DIR']
petsc_arch = petsc4py.lib.getPathArchPETSc()[1]

# Get PETSc int and scalar types
scalar_size = np.dtype(PETSc.ScalarType).itemsize
index_size = np.dtype(PETSc.IntType).itemsize

if index_size == 8:
    c_int_t = "int64_t"
    ctypes_index = ctypes.c_int64
elif index_size == 4:
    c_int_t = "int32_t"
    ctypes_index = ctypes.c_int32
else:
    raise RuntimeError(f"Cannot translate PETSc index size into a C type, index_size: {index_size}.")

if scalar_size == 8:
    c_scalar_t = "double"
    numba_scalar_t = numba.types.float64
elif scalar_size == 4:
    c_scalar_t = "float"
    numba_scalar_t = numba.types.float32
else:
    raise RuntimeError(
        f"Cannot translate PETSc scalar type to a C type, complex: {complex} size: {scalar_size}.")

# Load PETSc library via ctypes
petsc_lib_name = ctypes.util.find_library("petsc")
if petsc_lib_name is not None:
    petsc_lib_ctypes = ctypes.CDLL(petsc_lib_name)
else:
    try:
        petsc_lib_ctypes = ctypes.CDLL(os.path.join(petsc_dir, petsc_arch, "lib", "libpetsc.so"))
    except OSError:
        petsc_lib_ctypes = ctypes.CDLL(os.path.join(petsc_dir, petsc_arch, "lib", "libpetsc.dylib"))
    except OSError:
        print("Could not load PETSc library for CFFI (ABI mode).")
        raise

# Get the PETSc MatSetValuesLocal function via ctypes
MatSetValues_ctypes = petsc_lib_ctypes.MatSetValuesLocal
MatSetValues_ctypes.argtypes = (ctypes.c_void_p, ctypes_index, ctypes.POINTER(
    ctypes_index), ctypes_index, ctypes.POINTER(ctypes_index), ctypes.c_void_p, ctypes.c_int)
del petsc_lib_ctypes

# ffi = cffi.FFI()

def get_kernel(domain, form):
    ufcx_form, _, _ = ffcx_jit(domain.comm, form, form_compiler_params={"scalar_type": "double"})
    kernel = ufcx_form.integrals(0)[0].tabulate_tensor_float64
    return kernel

In [18]:
# Create mesh and define function space
# domain = mesh.create_rectangle(comm=MPI.COMM_WORLD,
#                             points=((0.0, 0.0), (L, W)), n=(1, 1),
#                             cell_type=mesh.CellType.triangle,)
# N = 2
# domain = mesh.create_unit_square(MPI.COMM_WORLD, N, N)#, mesh.CellType.quadrilateral

V = fem.VectorFunctionSpace(domain, ("Lagrange", 1))
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
DG0 = fem.FunctionSpace(domain, ('DG', 0))

left_facets = mesh.locate_entities_boundary(domain, dim=1, marker=lambda x: np.isclose(x[0], 0.0))
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=left_facets)
bc = fem.dirichletbc(value=fem.Constant(domain, (PETSc.ScalarType(0), PETSc.ScalarType(0))), dofs=dofs, V=V)  

# Unpack mesh and dofmap data
num_owned_cells = domain.topology.index_map(domain.topology.dim).size_local
num_cells = num_owned_cells + domain.topology.index_map(domain.topology.dim).num_ghosts
x_dofs = domain.geometry.dofmap.array.reshape(num_cells, 3)
x = domain.geometry.x
dofmap = V.dofmap.list.array.reshape(num_cells, 3).astype(np.dtype(PETSc.IntType))

mu = fem.Function(DG0)
lambda_ = fem.Function(DG0)

mu.vector.set(1)
lambda_.vector.set(beta)

def epsilon(u):
    return ufl.sym(ufl.grad(u))

def sigma(u):
    return lambda_*ufl.div(u)*ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

# Define variational problem
f = fem.Constant(domain, (PETSc.ScalarType(0), PETSc.ScalarType(-rho*g)))
b = ufl.inner(f, v)*ufl.dx

a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx

In [29]:
A = fem.petsc.create_matrix(fem.form(a))
A.zeroEntries()
# fem.petsc.assemble_matrix(A, fem.form(a), bcs=[bc])
# A.assemble()

rhs = fem.petsc.create_vector(fem.form(b))

# with rhs.localForm() as b_local:
#     b_local.set(0.0)
# fem.petsc.assemble_vector(rhs, fem.form(b))
# fem.apply_lifting(rhs, [fem.form(a)], bcs=[[bc]])
# rhs.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
# fem.set_bc(rhs, [bc])
# print(A[:,:])
# print(rhs[:])

# A.zeroEntries()

with rhs.localForm() as b_local:
    b_local.set(0.0)
    
map_c = domain.topology.index_map(domain.topology.dim)
num_cells = map_c.size_local + map_c.num_ghosts
# cells = np.arange(0, num_cells, dtype=np.int32)
N_dofs_element = V.element.space_dimension
N_sub_spaces = V.num_sub_spaces # V.dofmap.index_map_bs
dofmap = V.dofmap.list.array.reshape(num_cells, int(N_dofs_element/N_sub_spaces))

dofmap_tmp = (N_sub_spaces*np.repeat(dofmap, N_sub_spaces).reshape(-1, N_sub_spaces) + np.arange(N_sub_spaces)).reshape(-1, N_dofs_element).astype(np.dtype(PETSc.IntType))        

kernel_A = get_kernel(domain, a)
kernel_b = get_kernel(domain, b)

@numba.njit
def get_dofs_bc(pos, bc_dofs):
    dofs_bc = []
    for i, dof in enumerate(pos):
        if dof in bc_dofs:
            dofs_bc.append(i)
    return dofs_bc

bc_dofs = bc.dof_indices()[0]

@numba.njit
def assemble_ufc(A, b, mesh, dofmap, num_cells, set_vals, mode):
    v, x = mesh
    entity_local_index = np.array([0], dtype=np.intc)
    perm = np.array([0], dtype=np.uint8)
    geometry = np.zeros((3, 3))
    coeffs_A = np.array([beta, 1], dtype=PETSc.ScalarType)
    coeffs_b = np.zeros((9), dtype=PETSc.ScalarType) #!!!!!!!!!!!
    constants = np.array([0, -rho*g], dtype=PETSc.ScalarType) #np.array(C, dtype=PETSc.ScalarType)

    b_local = np.zeros(N_dofs_element, dtype=PETSc.ScalarType)
    A_local = np.zeros((N_dofs_element, N_dofs_element), dtype=PETSc.ScalarType)
    
    for cell in range(num_cells):
        pos = rows = cols = dofmap[cell, :]
        # print(rows)
        geometry[:] = x[v[cell, :], :]
        b_local.fill(0.)
        A_local.fill(0.)

        kernel_b(ffi.from_buffer(b_local), ffi.from_buffer(coeffs_b),
                 ffi.from_buffer(constants),
                 ffi.from_buffer(geometry), ffi.from_buffer(entity_local_index),
                 ffi.from_buffer(perm))
        b[pos] += b_local
        
        kernel_A(ffi.from_buffer(A_local), ffi.from_buffer(coeffs_A),
               ffi.from_buffer(constants),
               ffi.from_buffer(geometry), ffi.from_buffer(entity_local_index),
               ffi.from_buffer(perm))

        # print()
        for i in get_dofs_bc(pos, bc_dofs):
            A_local[i,:] = 0
            A_local[:,i] = 0
            # A_local[i,i] = 1./2. #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        # print('\n', A_local)
        set_vals(A, N_dofs_element, rows.ctypes, N_dofs_element, cols.ctypes, A_local.ctypes, mode)


assemble_ufc(A.handle, rhs.array, (x_dofs, x), dofmap_tmp, num_owned_cells, MatSetValues_ctypes, PETSc.InsertMode.ADD_VALUES)
A.assemble()
for i in bc.dof_indices()[0]:
    A[i,i] = 1
    # for j in np.arange(len(rhs.array)):
    #     print(i,j)
    #     # A[i,j] = 0
    #     A[j,i] = 0
    # A[:,i] = 0
A.assemble()

print(A[:,:])

fem.apply_lifting(rhs, [fem.form(a)], bcs=[[bc]])
rhs.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
fem.set_bc(rhs, [bc])
print(rhs[:])

solver = PETSc.KSP().create(A.getComm())
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

uh3 = fem.Function(V)
solver.solve(rhs, uh3.vector)
uh3.x.scatter_forward()

# print(bc.dof_indices())
# print(A[:,:])
# print(rhs[:])

[[  1.       0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       1.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       5.1625  -1.125   -5.       0.625    0.       0.    ]
 [  0.       0.      -1.125   16.3      0.5    -16.25     0.       0.    ]
 [  0.       0.      -5.       0.5      5.1625   0.       0.       0.    ]
 [  0.       0.       0.625  -16.25     0.      16.3      0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       1.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.       1.    ]]
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.66666667e-05
  0.00000000e+00 -1.33333333e-04  0.00000000e+00  0.00000000e+00]


In [30]:
print(uc.x.array[:])
print(uh.x.array[:])
print(uh2.x.array[:])
print(uh3.x.array[:])

[ 0.          0.         -0.00021879 -0.00395667  0.00017131 -0.00394433
  0.          0.        ]
[ 0.          0.         -0.00021879 -0.00395667  0.00017131 -0.00394433
  0.          0.        ]
[ 0.          0.         -0.00021879 -0.00395667  0.00017131 -0.00394433
  0.          0.        ]
[ 0.          0.         -0.00021879 -0.00395667  0.00017131 -0.00394433
  0.          0.        ]


In [23]:
uc.name = "Displacement"
uh.name = "linear solver"
uh2.name = "manual solver"
uh3.name = "ufc assembling"

with io.XDMFFile(MPI.COMM_WORLD, "solution_0.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5) as file:
    file.write_mesh(domain)

with io.XDMFFile(MPI.COMM_WORLD, "solution_0.xdmf", "a", encoding=io.XDMFFile.Encoding.HDF5) as file:
    file.write_function(uc)
    file.write_function(uh)
    file.write_function(uh2)
    file.write_function(uh3)

In [20]:
A_loc1 = np.array(
 [[ 1.75,0., -1.75,0.5, 0., -0.5  ],
 [ 0.,  0.625, 0.625, -0.625, -0.625, 0.],
 [-1.75,0.625, 2.375, -1.125, -0.625, 0.5 ],
 [ 0.5,-0.625, -1.125, 2.375, 0.625, -1.75 ],
 [ 0., -0.625, -0.625, 0.625, 0.625, 0.],
 [-0.5, 0., 0.5,-1.75,0., 1.75 ]])

A_loc2 = np.array( [[ 0.625, 0., -0.625, 0.625, 0., -0.625],
 [ 0., 1.75,0.5,-1.75 , -0.5, 0.,],
 [-0.625, 0.5, 2.375, -1.125, -1.75,0.625],
 [ 0.625, -1.75 , -1.125, 2.375, 0.5,-0.625],
 [ 0., -0.5,-1.75,0.5, 1.75,0.,],
 [-0.625, 0., 0.625, -0.625, 0., 0.625]])

# A_loc = np.array(
#   [[ 1.5, 0., -1.5, 0.5, 0., -0.5],
#  [ 0.,  0.5, 0.5, -0.5, -0.5, 0. ],
#  [-1.5, 0.5, 2., -1., -0.5, 0.5],
#  [ 0.5, -0.5, -1.,  2.,  0.5, -1.5],
#  [ 0.,, -0.5, -0.5, 0.5, 0.5, 0. ],
#  [-0.5, 0.,  0.5, -1.5, 0.,  1.5]])

# A_loc = np.array( [[ 0.625, 0., -0.625, 0.625, 0., -0.625],
#  [ 0., 1.75,0.5,-1.75 , -0.5, 0.,],
#  [-0.625, 0.5, 2.375, -1.125, -1.75,0.625],
#  [ 0.625, -1.75 , -1.125, 2.375, 0.5,-0.625],
#  [ 0., -0.5,-1.75,0.5, 1.75,0.,],
#  [-0.625, 0., 0.625, -0.625, 0., 0.625]])
 
S1 = np.array([[1,0, 0,0, 0,0, 0,0], 
               [0,1, 0,0, 0,0, 0,0], 
               [0,0, 1,0, 0,0, 0,0], 
               [0,0, 0,1, 0,0, 0,0], 
               [0,0, 0,0, 1,0, 0,0],
               [0,0, 0,0, 0,1, 0,0]])
               
S2 = np.array([[1,0, 0,0, 0,0, 0,0], 
               [0,1, 0,0, 0,0, 0,0], 
               [0,0, 0,0, 0,0, 1,0], 
               [0,0, 0,0, 0,0, 0,1], 
               [0,0, 0,0, 1,0, 0,0],
               [0,0, 0,0, 0,1, 0,0]])               
A1 = S1.T @ A_loc1 @ S1
A2 = S2.T @ A_loc2 @ S2
A1+A2

array([[ 2.375,  0.   , -1.75 ,  0.5  ,  0.   , -1.125, -0.625,  0.625],
       [ 0.   ,  2.375,  0.625, -0.625, -1.125,  0.   ,  0.5  , -1.75 ],
       [-1.75 ,  0.625,  2.375, -1.125, -0.625,  0.5  ,  0.   ,  0.   ],
       [ 0.5  , -0.625, -1.125,  2.375,  0.625, -1.75 ,  0.   ,  0.   ],
       [ 0.   , -1.125, -0.625,  0.625,  2.375,  0.   , -1.75 ,  0.5  ],
       [-1.125,  0.   ,  0.5  , -1.75 ,  0.   ,  2.375,  0.625, -0.625],
       [-0.625,  0.5  ,  0.   ,  0.   , -1.75 ,  0.625,  2.375, -1.125],
       [ 0.625, -1.75 ,  0.   ,  0.   ,  0.5  , -0.625, -1.125,  2.375]])